# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [ ]:
url = "https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84&sort=sale"
# this url returns a json, it is not browserable.

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

### imports

In [2]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

### data overview

In [4]:
# your code here

url = "https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84&sort=sale"

headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36'}

response = requests.get(url,headers=headers)
results = response.json()


flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])

results

{'total_count': 42005,
 'pagination': {'page_count': 501, 'current_page': 2, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'NI112B0CK-A15',
   'name': 'CLASSIC CORTEZ - Baskets basses - white/varsity red/varsity royal',
   'price': {'original': '84,95\xa0€',
    'promotional': '51,77\xa0€',
    'has_different_prices': True,
    'has_different_original_prices': False,
    'has_different_promotional_prices': True,
    'has_discount_on_selected_sizes_only': False},
   'url_key': 'nike-sportswear-classic-cortez-baskets-basses-ni112b0ck-a15',
   'media': [{'path': 'NI/11/2B/0C/KA/15/NI112B0CK-A15@12.jpg',
     'role': 'DEFAULT',
     'packet_shot': True}],
   'brand_name': 'Nike Sportswear',
   'is_premium': False,
   'flags': [{'key': 'discountRate',
     'value': 'Jusqu’à -39%',
     'tracking_value': 'discount rate'}],
   'product_group': 'shoe'},
  {'sku': 'SE212A01J-503',
   'name': 'SPINNAKER - Chaussures bateau - blue/brown',
   'price': {'original': '144,95\xa0€',
    'pro

In [5]:
print(flattened_data.shape)
flattened_data.head(2)

(1, 153)


,articles,articlesToShow,breadcrumbs,carouselTeaser,categoryTree,collection,contentPositions.entry-point-teasers,contentPositions.in-cat-carousel,contentPositions.in-cat-carousel-fullwidth,contentPositions.in-cat-carousel-mobile,...,total_article_count,total_count,upperInCatTeaser,variants.fullWidthCatalog,variants.hideCategories,variants.mobileLightFilters,variants.myBrandsFilter,variants.outwardTeaserCard,variants.premiumCatalog,wishlist
0,"[{'sku': 'NI112B0CK-A15', 'name': 'CLASSIC COR...",84,"[{'items': [{'label': 'Homme', 'url_key': 'hom...",None,"[{'label': 'Promotions', 'id': '9191', 'url_ke...",None,"[7, 14, 20, 26]",9,8,6,...,42009,42005,None,False,False,False,True,False,False,None


In [6]:
print(flattened_data1.shape)
flattened_data1.head(2)

(84, 15)


,brand_name,flags,is_premium,media,name,outfits,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sku,url_key
0,Nike Sportswear,"[{'key': 'discountRate', 'value': 'Jusqu’à -39...",False,[{'path': 'NI/11/2B/0C/KA/15/NI112B0CK-A15@12....,CLASSIC CORTEZ - Baskets basses - white/varsit...,NaN,False,True,True,False,"84,95 €","51,77 €",shoe,NI112B0CK-A15,nike-sportswear-classic-cortez-baskets-basses-...
1,Sebago,"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'SE/21/2A/01/J5/03/SE212A01J-503@1.j...,SPINNAKER - Chaussures bateau - blue/brown,NaN,False,True,True,False,"144,95 €","87,00 €",shoe,SE212A01J-503,sebago-spinnaker-chaussures-bateau-bleu-se212a...


In [7]:
total_pages = results['pagination']['page_count']
print(total_pages)

501


### main loop

In [10]:
# your code here
def get_data(url, total_pages, offset=84):
    df=pd.DataFrame()
    for i in range(total_pages):
        k=offset*i
        url = url + "&offset=" + str(k)
        headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36'}
        response = requests.get(url,headers=headers)
        results = response.json()
        flattened_data = json_normalize(results)
        flattened_data1 = json_normalize(flattened_data.articles[0])
        flattened_data1=flattened_data1.set_index('sku')
        df = df.append(flattened_data1)
    return df

url = "https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84"
df = get_data(url,total_pages)
print(df.shape)
df.head(2)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(42084, 18)


,amount,brand_name,flags,is_premium,media,name,outfits,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
sku,,,,,,,,,,,,,,,,,,
SE212A00T-Q11,NaN,Sebago,"[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'SE/21/2A/00/TQ/11/SE212A00T-Q11@12....,CLASSIC DAN - Mocassins - black,NaN,False,True,True,False,"199,95 €","110,00 €",shoe,https://ccp-et.metrigo.zalan.do/event/sbv?z=c1...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=c...,ccp,sebago-classics-mocassins-se212a00t-q11
TH342E038-Q11,NaN,The North Face,"[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'TH/34/2E/03/8Q/11/TH342E038-Q11@4.1...,Pantalon de survêtement - black,NaN,False,True,True,False,"85,00 €","76,50 €",clothing,https://ccp-et.metrigo.zalan.do/event/sbv?z=c1...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=c...,ccp,the-north-face-pantalon-de-survetement-black-t...


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

### trending brand

In [11]:
# your code here
df.brand_name.value_counts().head(1)

Nike Sportswear    7055
Name: brand_name, dtype: int64

### Highest Discount Products

In [12]:
#Our data is still text. Convert prices into numbers.
# your code
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

### Calculate Discounts

The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices). =? <br>
df['discounts']=df['price.promotional'].astype(float) / df['price.original'].astype(float)

In [28]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df['discounts']=df['price.promotional'].astype(float) / df['price.original'].astype(float)
df['discount_percent'] = 100 -((df['price.promotional'].astype(float) / df['price.original'].astype(float)) *100)

In [29]:
df1 = df[['brand_name','name','price.original','price.promotional','discounts','discount_percent','discount_amount']]
df1.sort_values('discount_percent',ascending=False).head(2)

,brand_name,name,price.original,price.promotional,discounts,discount_percent,discount_amount
sku,,,,,,,
TH342F05P-I11,The North Face,FANORAK - Veste coupe-vent - lupin liliac/whit...,114.95,33.95,0.295346,70.46542,81.0
TH342F05P-I11,The North Face,FANORAK - Veste coupe-vent - lupin liliac/whit...,114.95,33.95,0.295346,70.46542,81.0


### sum of discounts

In [30]:
df1.discounts.sum()

22804.80927791321